In [ ]:
import pandas as pd
import numpy as np  

In [ ]:
kurlamain = pd.read_csv('Data\Kurla_dataset.csv')
airportmain = pd.read_csv('Data\Airport_dataset.csv')
chakalamain = pd.read_csv('Data\Chakala_dataset.csv')

In [ ]:
display(kurlamain)
display(airportmain)
display(chakalamain)

In [ ]:
kurlamain.replace('None', np.nan, inplace=True)
kurlamain
airportmain.replace('None', np.nan, inplace=True)
airportmain
chakalamain.replace('None', np.nan, inplace=True)
chakalamain

Processing the missing data with "bfill" & "ffill"

In [ ]:
kurlafilled=kurlamain.fillna(method='ffill', inplace=False)
kurlafilled=kurlamain.fillna(method='bfill', inplace=False)
kurlafilled
airportfilled=airportmain.fillna(method='ffill', inplace=False)
airportfilled=airportmain.fillna(method='bfill', inplace=False)
airportfilled
chakalafilled=kurlamain.fillna(method='ffill', inplace=False)
chakalafilled=kurlamain.fillna(method='bfill', inplace=False)
chakalafilled

Processing the missing data with Interpolate

In [ ]:
kurlainter = kurlamain
kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= kurlainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].interpolate(method ='linear', limit_direction ='both')
kurlainter

In [ ]:
airportinter = airportmain
airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]= airportinter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].interpolate(method ='linear', limit_direction ='both')
airportinter

In [ ]:
chakalainter = chakalamain
chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']]= chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']].astype(float)
chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']]= chakalainter[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','Temp']].interpolate(method ='linear', limit_direction ='both')
chakalainter 

In [ ]:
kurlastd = kurlafilled
def determine_outlier_thresholds_std(dataframe, col_name):
    upper_boundary = dataframe[col_name].mean() + 3 * dataframe[col_name].std()
    lower_boundary = dataframe[col_name].mean() - 3 * dataframe[col_name].std()
    return lower_boundary, upper_boundary
def check_outliers_std(dataframe, col_name):
    lower_boundary, upper_boundary = determine_outlier_thresholds_std(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_boundary) | (dataframe[col_name] < lower_boundary)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_std(dataframe, cols, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_std(dataframe, col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_std(dataframe, col_name)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                if replace:
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_std(dataframe, col_name)
            data.append([outliers_, outliers_status,count, col_name, lower_limit, upper_limit])
    table = tabulate(data, headers=['Outlier (Previously)','Outliers','Count', 'Column','Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    print("Removing Outliers using 3 Standard Deviation")
    print(table)

replace_with_thresholds_std(kurlastd, kurlastd.columns,replace=False)

In [ ]:
kurlafilled[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']]=kurlafilled[['PM2.5','PM10','NO','NO2','SO2','CO','Ozone','RH','WS','WD','AT']].astype(float)
kurlaiqr = kurlafilled
kurlaiqr.drop(kurlaiqr.columns[[0,1]], axis=1, inplace=True)
kurlaiqr = kurlaiqr.apply(pd.to_numeric, errors='coerce')


In [ ]:
def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.25, th3=0.75):
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit

def check_outliers_iqr(dataframe, col_name):
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else: 
        return False

def replace_with_thresholds_iqr(dataframe,cols, th1=0.05, th3=0.95, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][cols].count()
                if replace: 
                    if lower_limit < 0:
                        # We don't want to replace with negative values, right!
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
                    else:
                        dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                        dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(dataframe, col_name)
            data.append([outliers_, outliers_status, count, col_name, lower_limit, upper_limit ])
    table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
    print("Removing Outliers using IQR")
    print(table)

replace_with_thresholds_iqr(kurlaiqr, kurlaiqr.columns)
# check_outliers_iqr(filled1, 'PM2.5')